In [3]:
import os 

os.environ["MLFLOW_TRACKING_URI"] = "https://dagshub.com/naveenmails814/End-to-End_DS_Project_1.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"] = "naveenmails814"
os.environ["MLFLOW_TRACKING_PASSWORD"] = "bcc8e73556386aeab1e35ff266f687ca3fb74045"

In [4]:
import os 
%pwd

'd:\\VS_code\\MLops\\11_End-to-End_Projects\\End-to-End_DS_Project_1\\research'

In [5]:
os.chdir("../")
%pwd

'd:\\VS_code\\MLops\\11_End-to-End_Projects\\End-to-End_DS_Project_1'

In [6]:
from dataclasses import dataclass
from pathlib import Path 

@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path 
    all_params: dict 
    metrics_file_path: Path 
    target_column: str 
    mlflow_uri: str

In [7]:
from src.dataScience.constants import * 
from src.dataScience.utils.common import read_yaml, create_directories, save_json

In [8]:
class ConfigurationManager:
    def __init__(
            self, 
            config_file_path = CONFIG_FILE_PATH,
            params_file_path = PARAMS_FILE_PATH, 
            schema_file_path = SCHEMA_FILE_PATH
            ):
        
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)
        self.schema = read_yaml(schema_file_path)   

        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path= config.model_path,
            all_params= params,
            metrics_file_path= config.metrics_file_path,
            target_column = schema.name,
            mlflow_uri= "https://dagshub.com/naveenmails814/End-to-End_DS_Project_1.mlflow"
            )
        return model_evaluation_config


In [9]:
import os 
import pandas as pd 
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from urllib.parse import urlparse 
import mlflow 
import mlflow.sklearn
import numpy as np
import joblib 

In [10]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    def eval_metrics(self, actual, pred):
        rmse = np.sqrt(mean_absolute_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2 
    
    def log_into_mlflow(self):

        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        X_test = test_data.drop([self.config.target_column], axis=1)
        y_test = test_data[[self.config.target_column]]

        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():

            predicted_qualities = model.predict(X_test)

            (rmse, mae, r2) = self.eval_metrics(y_test, predicted_qualities)

            # Saveing metrics as local 
            scores = {"rmse": rmse, "mae": mae, "r2": r2}
            save_json(path = Path(self.config.metrics_file_path), data=scores)

            mlflow.log_params(self.config.all_params)

            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("mae", mae)
            mlflow.log_metric( "r2", r2)

            if tracking_url_type_store != "file":
                mlflow.sklearn.log_model(model, "model", registered_model_name="ElasticNetModel")
            else: 
                mlflow.sklearn.log_model(model, "model")

In [11]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation = ModelEvaluation(config= model_evaluation_config)
    model_evaluation.log_into_mlflow()
except Exception as e:
    raise e

[2024-11-26 15:13:01,317: INFO: common: yaml file : config\config.yaml loaded successfully]
[2024-11-26 15:13:01,441: INFO: common: yaml file : params.yaml loaded successfully]
[2024-11-26 15:13:01,482: INFO: common: yaml file : schema.yaml loaded successfully]
[2024-11-26 15:13:01,499: INFO: common: Created directory at: artifacts]
[2024-11-26 15:13:01,562: INFO: common: Created directory at: artifacts/model_evaluation]


2024/11/26 15:13:35 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Successfully registered model 'ElasticNetModel'.
2024/11/26 15:13:40 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: ElasticNetModel, version 1
Created version '1' of model 'ElasticNetModel'.
2024/11/26 15:13:41 INFO mlflow.tracking._tracking_service.client: 🏃 View run gaudy-lynx-133 at: https://dagshub.com/naveenmails814/End-to-End_DS_Project_1.mlflow/#/experiments/0/runs/b47873e2960d42518e6a09427b35f7a0.
2024/11/26 15:13:41 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/naveenmails814/End-to-End_DS_Project_1.mlflow/#/experiments/0.
